In [4]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.linalg as la
from util import Trainer   
import networkx 
import time
import math
# from scipy.sparse import isspmatrix
# from scipy.special import expit as sigmoid
# from constants import INIT_WEIGHT_STD
from parameters import Parameters
import pickle
import csv
import os


def qsgd_quantize(x, d, is_biased):
    norm = np.sqrt(np.sum(np.square(x)))
    if norm <= 0.1: #norm cannot be zero
        norm = 0.1
    level_float = d * np.abs(x) / norm
    previous_level = np.floor(level_float)
    is_next_level = np.random.rand(*x.shape) < (level_float - previous_level)
    new_level = previous_level + is_next_level
    scale = 1
    if is_biased:
        n = len(x)
        scale = 1. / (np.minimum(n / d ** 2, np.sqrt(n) / d) + 1.)
    return scale * np.sign(x) * norm * new_level / d

def qsgd_quantize_1(x, d, is_biased):   # d quantization level
    norm = np.sqrt(np.sum(np.square(x)))
    if norm <= 0.1: #norm cannot be zero
        norm = 0.1
    level_float_1 = 2*d * x /  norm
    previous_level_1 = np.floor(level_float_1)
    previous_level_2 = np.where(previous_level_1 % 2 == 0, previous_level_1 - 1, previous_level_1)
    is_next_level_1 = (2*np.random.rand(*x.shape)) < (level_float_1 - previous_level_2)
    new_level_1 = previous_level_2 + (2*is_next_level_1)
    scale = 1
    if is_biased:
        n = len(x)
        scale = 1. / (np.minimum(n / d ** 2, np.sqrt(n) / d) + 1.)
    return scale *  norm * new_level_1 / (2*d)


def logistic_loss(x, l2):
    Z = np.log(1+np.exp(-np.prod(x, axis=0)))
    a = l2/2 * (la.norm(x, axis=0))**2  
    return Z+a

def logistic_gradient(x, l2, normalize=True):# x is a vector for an agent
    grad_1 = (- x[1]) / (1+np.exp(np.prod(x)))+l2*x[0]
    grad_2 = (- x[0]) / (1+np.exp(np.prod(x)))+l2*x[1]
    grad = [grad_1, grad_2]
    # if normalize:
    #     return grad
    return grad



class Parameters:
    """
    Parameters class used for all the experiments, redefine a string representation to summarize the experiment
    """

    def __init__(self,
                 t_0=0,
                 num_iterations=15000,
                 num_epoch=1,
                 lr_type='constant',
                 initial_lr=1,
                 regularizer= 1,
                 lr_alpha = 0.62,
                 lr_beta = 0.94,
                 lr_ck = 1,
            
                 epoch_decay_lr=None,
                 consensus_lr=None,
                 quantization="qsgd-unbiased",
                 # number of coordinates k in top-k or random-k quantization
                 coordinates_to_keep=None,
                 # number of levels in qsgd quantization
                 num_levels=10,
                 estimate='final',
                 name=None,
                 # number of machines
                 n_cores=5,
                 topology='connected',
                 method='sample-hold',
                 distribute_data=False,
                 # whether each machine gets random data or continuous set of data
                 # might not have any difference, depends on the dataset
                 split_data_strategy=None,
                 tau=None,
                 real_update_every=1,
                 random_seed=None,
                 split_data_random_seed=None,
                 ):
        # a lot of sanity checks to fail fast if we have inconsistent parameters
        assert num_epoch >= 0
        assert lr_type in ['constant', 'epoch-decay', 'decay', 'sample-hold', 'bottou',]

        if lr_type in ['constant', 'decay']:
            assert initial_lr > 0
        if lr_type == 'decay':
            assert initial_lr and tau
            assert regularizer > 0
        if lr_type == 'epoch-decay':
            assert epoch_decay_lr is not None
        if lr_type == 'sample-hold':    
            assert t_0 and initial_lr
            assert regularizer > 0
            
        if method in ['choco']:
            assert consensus_lr > 0
        else:
            assert consensus_lr is None

        assert quantization in ['full', 'top', 'random-biased', 'random-unbiased',
                                'qsgd-biased', 'qsgd-unbiased']
        if quantization == 'full':
            assert not coordinates_to_keep
        elif quantization in ['top', 'random-biased', 'random-unbiased']:
            assert coordinates_to_keep > 0
        else:
            assert num_levels > 0

        assert estimate in ['final', 'mean', 't+tau', '(t+tau)^2']

        assert n_cores > 0

        assert topology in ['connected','centralized', 'ring', 'torus', 'disconnected']

        assert method in ['sample-hold','choco', 'dcd-psgd', 'ecd-psgd', 'plain']
        if method in ['dcd-psgd', 'ecd-psgd']:
            assert quantization in ['random-unbiased', 'qsgd-unbiased']

        if not distribute_data:
            assert not split_data_strategy
        else:
            assert split_data_strategy in ['naive', 'random', 'label-sorted']
        
        self.t_0 = t_0
        self.num_epoch = num_epoch
        self.num_iterations=num_iterations
        self.lr_type = lr_type
        self.initial_lr = initial_lr  
        self.regularizer = regularizer   
        self.epoch_decay_lr = epoch_decay_lr
        self.lr_alpha = lr_alpha
        self.lr_beta = lr_beta
        self.lr_ck = lr_ck 
        self.consensus_lr = consensus_lr
        self.quantization = quantization
        self.coordinates_to_keep = coordinates_to_keep
        self.num_levels = num_levels
        self.estimate = estimate
        self.name = name
        self.n_cores = n_cores
        self.topology = topology
        self.tau = tau
        self.real_update_every = real_update_every
        self.random_seed = random_seed
        self.method = method
        self.distribute_data = distribute_data
        self.split_data_strategy = split_data_strategy
        self.split_data_random_seed = split_data_random_seed

    def __str__(self):
        if self.name:
            return self.name

        lr_str = self.lr_str()
        sparse_str = self.sparse_str()

        reg_str = ""
        if self.regularizer:
            reg_str = "-reg{}".format(self.regularizer)

        return "epoch{}-{}{}-{}-{}".format(self.num_epoch, lr_str, reg_str, sparse_str, self.estimate)

    def lr_str(self):
        lr_str = ""
        if self.lr_type == 'constant':
            lr_str = "lr{}".format(self.initial_lr)
        elif self.lr_type == 'decay':
            lr_str = "lr{}decay{}".format(self.initial_lr, self.epoch_decay_lr)
        elif self.lr_type == 'custom':
            lr_str = "lr{}/lambda*(t+{})".format(self.initial_lr, self.tau)
        elif self.lr_type == 'bottou':
            lr_str = "lr-bottou-{}".format(self.initial_lr)
        else:
            lr_str = "lr-{}".format(self.lr_type)

        return lr_str

    def sparse_str(self):
        sparse_str = self.quantization
        if self.quantization != 'full':
            sparse_str += "{}".format(self.coordinates_to_keep)
        return sparse_str

    def __repr__(self):
        return "Parameter('{}')".format(str(self))
    

class Switch_qdgd:
    
    def __create_mixing_matrix(self, topology, n_cores):
        if topology == 'ring':
            W = np.zeros(shape=(n_cores, n_cores))
            value = 1./3 if n_cores >= 3 else 1./2
            np.fill_diagonal(W, value)
            np.fill_diagonal(W[1:], value, wrap=False)
            np.fill_diagonal(W[:, 1:], value, wrap=False)
            W[0, n_cores - 1] = value
            W[n_cores - 1, 0] = value
            return W
        elif topology == 'centralized':
            W = np.ones((n_cores, n_cores), dtype=np.float64) / n_cores
            return W
        elif topology == 'connected':
            W = np.array([[0.6, 0, 0, 0.4, 0],[0.2, 0.8, 0, 0, 0], [0.2, 0.1, 0.4, 0, 0.3], [0, 0, 0, 0.6, 0.4],[0, 0.1, 0.6, 0, 0.3]])
            return W
        elif topology == 'disconnected':
            W = np.eye(n_cores)
            return W
        else:
            print('torus topology!')
            assert topology == 'torus'
            assert int(np.sqrt(n_cores)) ** 2 == n_cores
            G = networkx.generators.lattice.grid_2d_graph(int(np.sqrt(n_cores)),
                int(np.sqrt(n_cores)), periodic=True)
            W = networkx.adjacency_matrix(G).toarray()
            for i in range(0, W.shape[0]):
                W[i][i] = 1
            W = W / 5
            return W
        
    
    def __init__(self, params: Parameters):
        self.params=params
        # super().__init__()
        # super().__init__(params)
        self.x = None
        self.x_estimate = None
        self.W = self.__create_mixing_matrix(params.topology, params.n_cores) # topology='connected', n_cores=5
  
    def __quantize(self, x):
        # quantize according to quantization function
        # x: shape(num_features, n_cores)
        if self.params.quantization in ['qsgd-biased', 'qsgd-unbiased']:
            is_biased = (self.params.quantization == 'qsgd-biased')
            assert self.params.num_levels
            q = np.zeros_like(x) #creat q
            for i in range(0, q.shape[1]):
                q[:, i] = qsgd_quantize(x[:, i], self.params.num_levels, is_biased)
            return q
        if self.params.quantization == 'full':
            return x
        if self.params.quantization == 'top':
            q = np.zeros_like(x)
            k = self.params.coordinates_to_keep
            for i in range(0, q.shape[1]):
                indexes = np.argsort(np.abs(x[:, i]))[::-1]
                q[indexes[:k], i] = x[indexes[:k], i]
            return q
        
    def __quantize_1(self, x):
        
        if self.params.quantization in ['qsgd-biased', 'qsgd-unbiased']:
            is_biased = (self.params.quantization == 'qsgd-biased')
            assert self.params.num_levels
            q = np.zeros_like(x)
            for i in range(0, q.shape[1]):
                q[:, i] = qsgd_quantize_1(x[:, i], self.params.num_levels, is_biased)
            return q
        if self.params.quantization == 'full':
            return x
        if self.params.quantization == 'top':
            q = np.zeros_like(x)
            k = self.params.coordinates_to_keep
            for i in range(0, q.shape[1]):
                indexes = np.argsort(np.abs(x[:, i]))[::-1]
                q[indexes[:k], i] = x[indexes[:k], i]
            return q

    def lr(self, t_list, iteration):
        p = self.params
        t = iteration
        
        if t <= t_list[0]:
            lrep = p.initial_lr / (1 + p.regularizer * (t) ** p.lr_alpha)
            lret = p.initial_lr / (1 + p.regularizer * (t) ** p.lr_beta)
        elif t > t_list[-2] and t < t_list[-1]:
            lrep = p.initial_lr / (1 + p.regularizer * (t_list[-2]) ** p.lr_alpha)
            lret = p.initial_lr / (1 + p.regularizer * (t_list[-2]) ** p.lr_beta)
        elif t >= t_list[-1]:
            lrep = p.initial_lr / (1 + p.regularizer * (t) ** p.lr_alpha)
            lret = p.initial_lr / (1 + p.regularizer * (t) ** p.lr_beta)
            pre_idx = t_list[-1] + math.ceil(p.lr_ck/(p.initial_lr / (1 + p.regularizer * (t) ** p.lr_alpha)))
            t_list.append(pre_idx)
            
        return t_list, lrep, lret







    def fit(self):
        
        # y = np.copy(y_init)
        # num_samples, num_features = A.shape
        p = self.params
        num_samples, num_features = [1,2]
        # losses = np.zeros(p.num_epoch + 1)
        losses = np.zeros(p.num_iterations + 1)
        x_list = np.zeros((p.num_iterations+1, num_features, p.n_cores))
        data = []  
    
        INIT_WEIGHT_STD= 0.05
        # Initialization of parameters
        if self.x is None:
            self.x = np.random.normal(0, INIT_WEIGHT_STD, size=(num_features,))
            self.x = np.tile(self.x, (p.n_cores, 1)).T
            print(self.x)
            self.x_estimate = np.copy(self.x)
            self.x_hat = np.copy(self.x)
            if p.method == 'old':
                self.h = np.zeros_like(self.x)
                alpha = 1. / (A.shape[1] / p.coordinates_to_keep + 1)

        # splitting data onto machines, p.distribute_data == none
        if p.distribute_data:
            np.random.seed(p.split_data_random_seed)
            num_samples_per_machine = num_samples // p.n_cores
            if p.split_data_strategy == 'random':
                all_indexes = np.arange(num_samples)
                np.random.shuffle(all_indexes)
            elif p.split_data_strategy == 'naive':
                all_indexes = np.arange(num_samples)
            elif p.split_data_strategy == 'label-sorted':
                all_indexes = np.argsort(y)

            indices = []
            for machine in range(0, p.n_cores - 1):
                indices += [all_indexes[num_samples_per_machine * machine:\
                                        num_samples_per_machine * (machine + 1)]]
            indices += [all_indexes[num_samples_per_machine * (p.n_cores - 1):]]
            print("length of indices:", len(indices))
            print("length of last machine indices:", len(indices[-1]))
        else:
            num_samples_per_machine = num_samples
            indices = np.tile(np.arange(num_samples), (p.n_cores, 1))
        # should have shape (num_machines, num_samples)

        # if cifar10 or mnist dataset, then make it binary
        # if len(np.unique(y)) > 2:
        #     y[y < 5] = -1
        #     y[y >= 5] = 1
        # print("Number of different labels:", len(np.unique(y)))
        # epoch 0 loss evaluation
        seq_loss = logistic_loss(self.x, l2=0.1)
        losses[0] = np.sum(seq_loss) / len(seq_loss)                                         
        print(losses)
        # compute_loss_every = int(num_samples_per_machine / LOSS_PER_EPOCH)
        # all_losses = np.zeros(int(num_samples_per_machine * p.num_epoch / compute_loss_every) + 1)
        compute_loss_every = 1000
        
        losses = np.zeros(p.num_iterations + 1)
        

        train_start = time.time()
        np.random.seed(p.random_seed)
       
        

        t_list = [p.t_0,]

        pre_idx = p.t_0 + math.ceil(p.lr_ck/(p.initial_lr / (1 + p.regularizer * (p.t_0) ** p.lr_alpha)))
        t_list.append(pre_idx)
        lrep_list = []
        lret_list = []
        
        for t in np.arange(p.num_iterations):
            # for iteration in range(num_samples_per_machine):
                #t = epoch * num_samples_per_machine + iteration
                loss = logistic_loss(self.x, l2=0.1)
                x_list[t] = self.x
                losses[t] = np.sum(loss) / len(loss) 
                
#                 if t % compute_loss_every == 0:
#                     print('iteration {} x{} loss {} losses {}'.format( t, self.x, 
#                          loss, losses[t]))
                       
#                     if np.isinf(losses[t]) or np.isnan(losses[t]):
#                         print("finish training")
#                         break

                # lr = self.lr(epoch, iteration, num_samples_per_machine, num_features)
                t_list, lrep, lret = self.lr (t_list, t)   
                # print('lrep', lrep, 'lret', lret)
                lrep_list.append([lrep])
                lret_list.append([lret])
                
                # if t in t_list:
                if t % 10 == 0:
                    
                    # data.append([t, self.x, loss, losses[t]])
                    # data.append([t] + self.x.ravel().tolist() + [loss, losses[t]])
                    y=[]
                    for i in range(len(self.x)):
                        for j in range(len(self.x[i])):
                            y.append(self.x[i][j])
                    # print([t] + y + [loss, losses[t]])
                    
                    data.append([t] + y + [loss, losses[t]])
                    # data.append( y )
                    # data.append( losses[t] )
                   
                    
                    
                    # print(data)


                    print('iteration {} x1{} x2{} loss {} losses {}'.format( t, self.x[0], self.x[1], 
                         loss, losses[t]))
                       
                    if np.isinf(losses[t]) or np.isnan(losses[t]):
                        print("finish training")
                        break

                # Gradient step
                x_plus = np.zeros_like(self.x)
                for machine in range(0, p.n_cores):
                    # sample_idx = np.random.choice(indices[machine])
                    # a = A[sample_idx]
                    x = self.x[:, machine]
                    minus_grad = logistic_gradient (x, l2=0.1, normalize=True)   
                    
                    # minus_grad = y[sample_idx] * a * sigmoid(-y[sample_idx] * a.dot(x).squeeze())
                    # if isspmatrix(a):
                    #     minus_grad = minus_grad.toarray().squeeze(0)
                    # if p.regularizer:
                    #     minus_grad -= p.regularizer * x
                    x_plus[:, machine] = (1-lrep) * x  - [lret * grad for grad in minus_grad]

                # Communication step
                if p.method == "plain":
                    self.x = (self.x + x_plus).dot(self.W)
                if p.method == "choco":
                    x_plus += self.x
                    self.x = x_plus + p.consensus_lr * self.x_hat.dot(self.W - np.eye(p.n_cores))
                    quantized = self.__quantize(self.x - self.x_hat)
                    self.x_hat += quantized
                elif p.method == 'dcd-psgd':
                    x_plus += self.x.dot(self.W)
                    quantized = self.__quantize(x_plus - self.x)
                    self.x += quantized
                elif p.method == 'ecd-psgd':
                    x_plus += self.x_hat.dot(self.W)
                    z = (1 - 0.5 * (t + 1)) * self.x + 0.5 * (t + 1) * x_plus
                    quantized = self.__quantize(z)
                    self.x = np.copy(x_plus)
                    self.x_hat = (1 - 2. / (t + 1)) * self.x_hat + 2./(t + 1) * quantized
                elif p.method == 'sample-hold':
                    if t % 2 == 0:
                        quantized = self.__quantize(self.x)
                    else:
                        quantized = self.__quantize_1(self.x)

                    self.x = x_plus + lrep * (quantized).dot(self.W)
                    
                
                csv_file = 'optimization_log.csv'
                write_header = not os.path.isfile(csv_file)

        with open(csv_file, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            # if write_header:
                # writer.writerow(['Iteration', 'x', 'Loss', 'Total Loss'])
            writer.writerows(data)

        print("Training took: {}s".format(time.time() - train_start))

        return losses, x_list
    
# switch_qsdg = Switch_qdgd (t_0=2700, num_iterations=10000,lr_alpha = 0.64,lr_beta = 0.94,lr_ck = 1)
p_param=Parameters(t_0=10, num_iterations=8000, lr_alpha = 0.62, lr_beta = 0.94, lr_ck = 3, num_levels=10)
switch_qsdg = Switch_qdgd (p_param)
losses, x_list = switch_qsdg.fit()

[[ 0.016719    0.016719    0.016719    0.016719    0.016719  ]
 [-0.05097596 -0.05097596 -0.05097596 -0.05097596 -0.05097596]]
[0.69371731 0.         0.         ... 0.         0.         0.        ]
iteration 0 x1[0.016719 0.016719 0.016719 0.016719 0.016719] x2[-0.05097596 -0.05097596 -0.05097596 -0.05097596 -0.05097596] loss [0.69371731 0.69371731 0.69371731 0.69371731 0.69371731] losses 0.6937173087162906
iteration 10 x1[-0.04705415 -0.045097   -0.04926223 -0.04982801 -0.04984705] x2[-0.04959002 -0.04912621 -0.05460729 -0.05147059 -0.05388146] loss [0.69221482 0.69226243 0.69207348 0.69212227 0.69207456] losses 0.6921495114508799
iteration 20 x1[-0.07370185 -0.07377986 -0.07487756 -0.07612995 -0.07640015] x2[-0.07621936 -0.07655177 -0.07876462 -0.07591929 -0.07795281] loss [0.69090444 0.69089236 0.6907932  0.69083947 0.69076949] losses 0.6908397920853562
iteration 30 x1[-0.10052069 -0.10243477 -0.10599537 -0.09978083 -0.10393153] x2[-0.10478392 -0.10554329 -0.10615607 -0.10477659 -0